In [136]:
from datasets import load_dataset

sentiment_dataset = load_dataset("sepidmnorozy/Korean_sentiment")
# label column name 변경 (-> labels)
sentiment_dataset = sentiment_dataset.rename_column("label", "labels")

In [137]:
print(sentiment_dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 1333
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 2667
    })
})


In [140]:
print(sentiment_dataset["train"][0])
print(type(sentiment_dataset["train"][0]))

{'labels': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}
<class 'dict'>


In [141]:
"""
토크나이저 불러오기
"""
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

In [142]:
"""
토큰화 함수 정의
"""

def tokenize_funciton(sample):
    return tokenizer(
        sample["text"],  # 텍스트 데이터
        # padding=True,  # 패딩 적용
        padding="max_length",  # 최대 길이로 패딩
        truncation=True,  # 최대 길이를 넘어가는 데이터는 자름
        return_tensors="pt"  # 파이토치 텐서로 반환
    )

In [143]:
"""
토크나이저 사용하기
"""

tokenized_sentiment_dataset = sentiment_dataset.map(
    tokenize_funciton,
    batched=True,  # 병렬 처리 활성화
    remove_columns=["text"])  # 원본 데이터에서 text 컬럼 제거 (메모리 최적화)

In [144]:
"""
토크나이저가 적용된 데이터 셋 확인하기
"""
print(tokenized_sentiment_dataset)
print(type(tokenized_sentiment_dataset))

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1333
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2667
    })
})
<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
"""
데이터 셋 분리해서 준비하기
"""

# 학습용 데이터셋
train_dataset = tokenized_sentiment_dataset["train"]

# 검증용 데이터셋
validation_dataset = tokenized_sentiment_dataset["validation"]

# 테스트용 데이터셋
test_dataset = tokenized_sentiment_dataset["test"]

In [ ]:
"""
분리된 데이터 셋 확인하기
"""

# 학습용 데이터셋 확인
print(train_dataset[0])

# 검증용 데이터셋 확인
print(validation_dataset[0])

# 테스트용 데이터셋 확인
print(test_dataset[0])

In [ ]:
from transformers import TrainingArguments

# 하이처 파라미터 지정
training_args = TrainingArguments(
    output_dir="../results",  # 출력 디렉토리
    num_train_epochs=3,  # 학습할 에포크 수
    eval_strategy="steps",  # 평가 전략 : 에포크마다 검증 데이터셋에 대한 평가 지표 출력
    eval_steps=500,  # 500 스텝마다 평가
    per_device_train_batch_size=8,  # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,  # warmup 스텝 수
    weight_decay=0.01,  # 가중치 감쇠
    remove_unused_columns=False,  # 사용하지 않는 컬럼 제거
)

In [ ]:
"""
예측값과 실제 레이블 간의 정확도 계산
"""

from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": float(
            accuracy_score(labels, predictions)
        )
    }

In [ ]:
"""
BERT 분류기 사용하기
"""

from transformers import BertForSequenceClassification

clf_model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)

In [ ]:
"""
분류기 학습시키기
"""

from transformers import TrainingArguments

# 하이처 파라미터 지정
clf_training_args = TrainingArguments(
    output_dir="../clf_results",  # 출력 디렉토리
    num_train_epochs=3,  # 학습할 에포크 수
    eval_strategy="steps",  # 평가 전략 : 에포크마다 검증 데이터셋에 대한 평가 지표 출력
    eval_steps=500,  # 500 스텝마다 평가
    per_device_train_batch_size=8,  # 학습에 사용할 배치
    # 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,  # warmup 스텝 수
    weight_decay=0.01,  # 가중치 감쇠
    remove_unused_columns=False,  # 사용하지 않는 컬럼 제거
)

In [ ]:
"""
clf Trainer 불러오기
"""
from transformers import Trainer

clf_trainer = Trainer(
    model=clf_model,
    args=clf_training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
"""
mac gpu 사용하기
"""

import torch

print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}")

device = torch.device("mps")

clf_model.to(device)

In [ ]:
"""
clf Trainer 학습시키기
"""

clf_trainer.train()
